In [ ]:
#install libraries
!pip install tensorflow-gpu==1.15.2
!pip install keras==2.0.8
!pip install python-chess==0.25.1 
!pip install profilehooks
!pip install numpy
!pip install pyperclip
!pip install h5py

     |████████████████████████████████| 411.0MB 25kB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
     |████████████████████████████████| 3.8MB 28.1MB/s 
     |████████████████████████████████| 512kB 44.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=e48f0b7b004151eb0466dd8c85053e877c2e425c8348d7a0707a7e1fb0ca02a9
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorboard~=2.4, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorflow-estimator<2.5.0,>=2.4.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing instal

In [52]:
#import libraries
import tensorflow as tf
import keras
import chess
import numpy as np
import h5py
from tensorflow.keras import models, layers

In [102]:
squares_index = {
  'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7
}

def square_to_index(square):
    letter = chess.square_name(square)
    return 8 - int(letter[1]), squares_index[letter[0]]

def split_dims(board):
  # this is the 3d matrix
    board3d = np.zeros((14, 8, 8), dtype=np.int8)

  # here we add the pieces's view on the matrix
    for piece in chess.PIECE_TYPES:
        for square in board.pieces(piece, chess.WHITE):
            idx = np.unravel_index(square, (8, 8))
            board3d[piece - 1][7 - idx[0]][idx[1]] = 1
        for square in board.pieces(piece, chess.BLACK):
            idx = np.unravel_index(square, (8, 8))
            board3d[piece + 5][7 - idx[0]][idx[1]] = 1

  # add attacks and valid moves too
  # so the network knows what is being attacked
    aux = board.turn
    board.turn = chess.WHITE
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        board3d[12][i][j] += 1
    board.turn = chess.BLACK
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        board3d[13][i][j] += 1
    board.turn = aux
    return board3d

def move_dims(move):
  board3d = np.zeros((2,8,8), dtype= np.int8)
  i = squares_index[move[0]]
  j = 8 - int(move[1])
  board3d[0][j][i] = 1
  i = squares_index[move[2]]
  j = 8 - int(move[3])
  board3d[1][j][i] = 1
  return board3d


In [39]:
def BoardBeforeMove(board, move):
  dims = split_dims(board)
  UCI = chess.Move.uci(board.parse_san(move))
  board.push_uci(UCI)
  return dims, move_dims(UCI), UCI

In [33]:
def DataLoad():
  import pandas as pd
  dataframe = pd.read_csv('games.csv')
  dataframe = dataframe[dataframe['victory_status']== 'mate']
  dataframe_black = dataframe[dataframe['winner'] == 'black']
  dataframe_white = dataframe[dataframe['winner'] == 'white']
  return np.array(dataframe_white['moves']), np.array(dataframe_black['moves'])




In [40]:
def moves(game, type):
  Steps = game.split(' ')
  board = chess.Board()
  game = []
  moves = []
  UCIs = []
  i = 0
  for step in Steps:
    if (type == "white"):
      if (i % 2) == 0: 
        dims, UCI_board, UCI = BoardBeforeMove(board,step)
        moves.append(UCI_board)
        game.append(dims)
        UCIs.append(UCI)
      else:
        a, b, c = BoardBeforeMove(board,step)
    else:
      if (i % 2) != 0:
        dims, UCI_board, UCI = BoardBeforeMove(board,step)
        moves.append(UCI_board)
        game.append(dims)
        UCIs.append(UCI)
      else:
        a, b, c = BoardBeforeMove(board,step)
    i += 1
  return np.array(game), np.array(moves), np.array(UCIs)


In [103]:
def GetData():
  WhiteWin = []
  WhiteMoves = []
  WhiteUCI = []
  BlackWin = []
  BlackMoves = []
  BlackUCI = []
  wgames, bgames = DataLoad()
  for game in wgames:
    WW, WM, WU = moves(game, "white")
    WhiteWin.append(WW)
    WhiteMoves.append(WM)
    WhiteUCI.append(WU)
  for game in bgames:
    BW, BM, BU = moves(game, "black")
    BlackWin.append(BW)
    BlackMoves.append(BM)
    BlackUCI.append(BU)
  
  return WhiteWin, WhiteMoves, WhiteUCI, BlackWin, BlackMoves, BlackUCI

WX,WY,WU,BX,BY,BU = GetData()

In [104]:
TrainWX = []
TrainWY = []
TrainBX = []
TrainBY = []
for game, move in zip(WX,WY):
  for i in range(len(game)):
    TrainWX.append(game[i].reshape(8,8,14))
    TrainWY.append(move[i].reshape(1,128))

TrainWX = np.array(TrainWX)
TrainWY = np.array(TrainWY)


for game, move in zip(BX,BY):
  for i in range(len(game)):
    TrainBX.append(game[i].reshape(8,8,14))
    TrainBY.append(move[i].reshape(1,128))

TrainBX = np.array(TrainBX)
TrainBY = np.array(TrainBY)


In [116]:
model = models.Sequential()
model.add(layers.ZeroPadding2D(padding=(1,1), input_shape=(8, 8, 14)))
model.add(layers.Conv2D(32, (3, 3), strides = (1,1), activation='sigmoid'))
model.add(layers.ZeroPadding2D(padding=(1,1)))
model.add(layers.AveragePooling2D((2, 2)))
model.add(layers.ZeroPadding2D(padding=(1,1)))
model.add(layers.Conv2D(64, (3, 3), activation='sigmoid'))
model.add(layers.ZeroPadding2D(padding=(1,1)))
model.add(layers.AveragePooling2D((2, 2)))
model.add(layers.ZeroPadding2D(padding=(1,1)))
model.add(layers.Conv2D(64, (3, 3), activation='sigmoid'))
model.add(layers.Flatten())
model.add(layers.Dense(128,activation='softmax'))


In [117]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_68 (ZeroPaddi (None, 10, 10, 14)        0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 8, 8, 32)          4064      
_________________________________________________________________
zero_padding2d_69 (ZeroPaddi (None, 10, 10, 32)        0         
_________________________________________________________________
average_pooling2d (AveragePo (None, 5, 5, 32)          0         
_________________________________________________________________
zero_padding2d_70 (ZeroPaddi (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 5, 5, 64)          18496     
_________________________________________________________________
zero_padding2d_71 (ZeroPaddi (None, 7, 7, 64)        

In [119]:
model.compile(optimizer='adam',
              loss = tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

history = model.fit(TrainWX, TrainWY, epochs = 10)

Epoch 1/10
3385/3385 [==============================] - 46s 13ms/step - loss: 0.0153 - accuracy: 0.0226
Epoch 2/10
3385/3385 [==============================] - 46s 13ms/step - loss: 0.0153 - accuracy: 0.0188
Epoch 3/10
3385/3385 [==============================] - 46s 13ms/step - loss: 0.0153 - accuracy: 0.0169
Epoch 4/10
3385/3385 [==============================] - 46s 14ms/step - loss: 0.0153 - accuracy: 0.0188
Epoch 5/10
3385/3385 [==============================] - 46s 13ms/step - loss: 0.0153 - accuracy: 0.0224
Epoch 6/10
3385/3385 [==============================] - 46s 14ms/step - loss: 0.0153 - accuracy: 0.0143
Epoch 7/10
3385/3385 [==============================] - 46s 14ms/step - loss: 0.0153 - accuracy: 0.0171
Epoch 8/10
3385/3385 [==============================] - 46s 14ms/step - loss: 0.0153 - accuracy: 0.0129
Epoch 9/10
3385/3385 [==============================] - 46s 14ms/step - loss: 0.0153 - accuracy: 0.0103
Epoch 10/10
3385/3385 [==============================] - 46s 14m

In [155]:
prediction = model.predict(TrainWX[:1])


In [154]:
def moveEval(pred, move):
  UCI = str(move)
  i = squares_index[UCI[0]]
  j = 8 - int(UCI[1])
  k = squares_index[UCI[2]]
  l = 8 - int(UCI[1])
  return pred[0][j][i]*pred[1][l][k]


def Decision(Board, model):
  X = []
  X.append(split_dims(Board).reshape(8,8,14))
  X= np.array(X)
  prediction = model.predict(X[:1])
  curr_move = ""
  max = 0
  prediction = prediction.reshape(2,8,8)
  for move in board.legal_moves:
    curr = moveEval(prediction,move)
    if curr > max:
      curr_move = move
      max = curr
  return curr_move



move = Decision(board,model)
print(move)

e2e3


In [162]:
model.save("White_weights.h5")

In [166]:
WhiteModel = models.load_model("White_weights.h5")
move = Decision(board,WhiteModel)
print(move)

e2e3
